In [ ]:
from cobra import Model, Reaction, Metabolite
from cobra.io.sbml import read_sbml_model, write_sbml_model
from riptide import *
import pandas as pd
from copy import *
from difflib import *
import numpy as np 
from pathlib import Path
import glob

In [ ]:
from Bio import SeqIO
from Bio.KEGG import REST
from Bio.KEGG.KGML import KGML_parser
import pandas as pd
from cobra import Model, Reaction, Metabolite
from riptide import *
import pandas as pd
from copy import *
from cobra.medium import minimal_medium
from difflib import *

## BV+ Cervicovaginal Fluid

Cervicovaginal fluid [metabolomics](https://journals.asm.org/doi/10.1128/mBio.00204-15) comparing BV positive patients to BV negative patients metabolome

Specifically metabolites that are >1 ratio (negative vs. positive) & signficant based on Welch's two sample t-test

#### Accounted for:
myristoleate, kynurenate, pentadecanoate, 2-O-methylguanosine, 3-dephosphocoenzyme A, N2,N2-dimethylguanine, flavin adenine dinucleotide (FAD), Alanine, Valine, N-acetylalanine, 3-methyl-2-oxovalerate, 3-methyl-2-oxobutyrate, tricarballylate, 4-methyl-2-oxopentanoate, N-acetylneuraminate, citrulline, 2-aminobutyrate, phenylacetate, sarcosine (N-Methylglycine), nicotinate, palmitoyl ethanolamide, threitol, galactose, N6-acetyllysine, 1-phenylethanamine, alpha-hydroxyisocaproate, 2-Hydroxybutyrate, succinate, 2-hydroxyglutarate, 3-phenylpropionate (hydrocinnamate), thymine, agmatine, N-acetylputrescine, 4-hydroxybutyrate, tyramine, putrescine, deoxycarnitine, tryptamine, cadaverine, 5-aminovalerate, 4-Hydroxyphenylacetate, pipecolate

#### Not accounted for:
pelargonate, indolepropionate, myristate, 13-methylmyristic acid, N-acetylvaline, N-acetylphenylalanine, N-acetylleucine, N-acetylglutamate, 13-HODE, oleic ethanolamide, N-acetylaspartate (NAA), 12-HETE, alpha-hydroxyisovalerate, 2-hydroxy-3-methylvalerate, 3-(4-hydroxyphenyl)propionate

In [ ]:
cervicovaginal_cmpds = ["cpd05237",
                        "cpd01182",
                        "cpd15622",
                        "cpd34555",
                        "cpd00655",
                        "cpd28562",
                        "cpd00015",
                        "cpd00035",
                        "cpd00156",
                        "cpd33748",
                        "cpd00508",
                        "cpd00123",
                        "cpd16654",
                        "cpd00200",
                        "cpd00232",
                        "cpd00274",
                        "cpd01573",
                        "cpd00430",
                        "cpd00183",
                        "cpd00218",
                        "cpd16300",
                        "cpd17172",
                        "cpd00108",
                        "cpd01770",
                        "cpd34388",
                        "cpd33351",
                        "cpd03561",
                        "cpd00036",
                        "cpd02041",
                        "cpd03343",
                        "cpd00151",
                        "cpd00152",
                        "cpd01758",
                        "cpd00728",
                        "cpd00374",
                        "cpd00118",
                        "cpd00870",
                        "cpd00318",
                        "cpd01155",
                        "cpd00339",
                        "cpd00489",
                        "cpd00323"]

## Identifying existing exchange reactions & adding missing exchange reactions

Create list of metabolite ids

In [ ]:
media_exchange_ids = ["EX_"+ x + "_e" for x in cervicovaginal_cmpds]

In [ ]:
def minMedia(model):
    modelOutput = deepcopy(model)
    
    # create a list of existing exchange reactions
    model_exchange_ids = []
    for exchange in modelOutput.exchanges:
        model_exchange_ids.append(exchange.id)
            
    # Identify missing metabolites
    exchange_difference = [x for x in media_exchange_ids if x not in model_exchange_ids]
    missing_metabolites = [x[3:-2] for x in exchange_difference]
    
    # Add missing extracellular metabolites
    for cpd in missing_metabolites:
        cytosol = (cpd + "_c")
        for metabo in modelOutput.metabolites:
            if cytosol in metabo.id:
                metabolite = Metabolite(cpd + "_e")
                metabolite.name = modelOutput.metabolites.get_by_id(cytosol).name
                metabolite.formula = modelOutput.metabolites.get_by_id(cytosol).formula
                metabolite.compartment = "extracellular"
                modelOutput.add_metabolites([metabolite])
                
    # Add missing exchange reactions
    for metabolite in missing_metabolites:
        cytosol = (metabolite + "_c")
        for metabo in modelOutput.metabolites:
            if cytosol in metabo.id:
                reaction = Reaction('EX_' + metabolite + "_e")
                reaction.name = modelOutput.metabolites.get_by_id(metabolite+"_e").name + 'exchange'
                reaction.subsystem = 'exchange'
                reaction.lower_bound = 0 
                reaction.upper_bound = 1000
                reaction.add_metabolites({modelOutput.metabolites.get_by_id(metabolite+"_e"):-1.0})
                modelOutput.add_reactions([reaction])
                
    #Create list of minimal media reactions 
    min_media = minimal_medium(modelOutput, 0.2, open_exchanges=True)
    minmedia_open_exchanges = list(min_media.index)
    
    return minmedia_open_exchanges

In [ ]:
minmedia = []
file_list = glob.glob("reconstructions/*.sbml")
for old in file_list:
    model=read_sbml_model(old, low_memory = False)
    model_update = minMedia(model)
    min_exchanges = model_update
    minmedia.append(min_exchanges)

minmedia = set([item for sublist in minmedia for item in sublist])

In [ ]:
media_exchange_ids_update = list(set(media_exchange_ids + list(minmedia)))

In [ ]:
def updateModel(model):
    modelOutput = deepcopy(model)
    
    # create a list of existing exchange reactions
    model_exchange_ids = []
    for exchange in modelOutput.exchanges:
        media_exchange_ids_update.append(exchange.id)
            
    # Identify missing metabolites
    exchange_difference = [x for x in media_exchange_ids_update if x not in model_exchange_ids]
    missing_metabolites = [x[3:-2] for x in exchange_difference]
    
    # Add missing extracellular metabolites
    for cpd in missing_metabolites:
        cytosol = (cpd + "_c")
        for metabo in modelOutput.metabolites:
            if cytosol in metabo.id:
                metabolite = Metabolite(cpd + "_e")
                metabolite.name = modelOutput.metabolites.get_by_id(cytosol).name
                metabolite.formula = modelOutput.metabolites.get_by_id(cytosol).formula
                metabolite.compartment = "extracellular"
                modelOutput.add_metabolites([metabolite])
                
    # Add missing exchange reactions
    for metabolite in missing_metabolites:
        cytosol = (metabolite + "_c")
        for metabo in modelOutput.metabolites:
            if cytosol in metabo.id:
                reaction = Reaction('EX_' + metabolite + "_e")
                reaction.name = modelOutput.metabolites.get_by_id(metabolite+"_e").name + 'exchange'
                reaction.subsystem = 'exchange'
                reaction.lower_bound = 0 
                reaction.upper_bound = 1000
                reaction.add_metabolites({modelOutput.metabolites.get_by_id(metabolite+"_e"):-1.0})
                modelOutput.add_reactions([reaction])
  
    return modelOutput

In [ ]:
file_list = glob.glob("reconstructions/*sbml")
for old in file_list:
    model=read_sbml_model(old, low_memory=False)
    model_update = updateModel(model)
    name = old.lstrip('reconstructions/')
    name2 = name.rstrip(".sbml")
    write_sbml_model(model_update, "update_reconstructions/" + name2 + ".sbml")

In [ ]:
def changeMedia(model, media, limEX=[]):
    modelOutput = deepcopy(model)

    # Set the new media conditions
    for ex in modelOutput.exchanges:
        ex.upper_bound = 1000
        ex.lower_bound = 0
                               
    # BV+ Media + Minimal Media
    if media == 1:
        for exchange in modelOutput.reactions:
            if exchange.id in media_exchange_ids_update:
                exchange.lower_bound = -1000
   
    elif media == 0:
        print('all exchange bounds set to [0,1000]')
            
    else:
        print('unrecognized media condition. Please enter 1 for BV + media')
   
    return(modelOutput) 

# Output models with media constraints

In [ ]:
file_list = glob.glob("reconstructions/*.sbml")
for seq in file_list:
    model = cobra.io.read_sbml_model(seq)
    model_output = changeMedia(model, 1)
    print(model_output.optimize())
    name = seq.lstrip('reconstructions/')
    name2 = name.rstrip(".sbml")
    cobra.io.write_sbml_model(model_output, "BV+_context/" + name2 + ".sbml")